In [1]:
import pandas as pd
import numpy as np
import re
import gc
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
df_train_ident = pd.read_csv('data/train_identity.csv.gz')
df_train_transact = pd.read_csv('data/train_transaction.csv.gz')
df_test_ident = pd.read_csv('data/test_identity.csv.gz')
df_test_transact = pd.read_csv('data/test_transaction.csv.gz')

# Using TransactionID as index for both data.
df_train_ident.set_index('TransactionID',inplace=True)
df_train_transact.set_index('TransactionID',inplace=True)
df_test_ident.set_index('TransactionID',inplace=True)
df_test_transact.set_index('TransactionID',inplace=True)


In [3]:
df_train = pd.merge(df_train_transact, df_train_ident, on='TransactionID', how='left')
df_test = pd.merge(df_test_transact, df_test_ident, on='TransactionID', how='left')

In [4]:
def preprocessing(line):
    line = str(line).lower()
    line = re.sub(r"[^\w\s]", '_', line)
    line = re.sub(r"\s+", ' ', line)
    return line

In [5]:
miss_val_threshold = 0.25

col_to_del = []

for c in df_train.columns:
    if df_train[c].isnull().sum() > df_train.shape[0]*miss_val_threshold:
        col_to_del.append(c)

for c in df_test.columns:
    if df_train[c].isnull().sum() > df_test.shape[0]*miss_val_threshold:
        if c not in col_to_del:
            col_to_del.append(c)
        
col_to_del.append('TransactionID')
col_to_del.append('TransactionDT')

In [6]:
df_train.drop(columns=col_to_del, inplace = True)
df_test.drop(columns=col_to_del, inplace = True)

In [7]:
df_train.fillna(-999, inplace= True)
df_test.fillna(-999, inplace= True)

In [8]:
# before optimization
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 180 entries, isFraud to V321
dtypes: float64(174), int64(2), object(4)
memory usage: 815.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 0 to 506690
Columns: 179 entries, TransactionAmt to V321
dtypes: float64(174), int64(1), object(4)
memory usage: 695.8+ MB


In [9]:
col_int16 = ['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2',
           'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14',
           'D1', 'D10', 'D15', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
           'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30',
           'V31', 'V32', 'V33', 'V34', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59',
           'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 
           'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79',
           'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89',
           'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99',
           'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109',
           'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119',
           'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129',
           'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V279',
           'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289',
           'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299',
           'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309',
           'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319',
           'V320', 'V321']

In [10]:
for c in col_int16:
    df_train[c] = df_train[c].astype(np.int16)
    df_test[c] = df_test[c].astype(np.int16)

In [11]:
# one-hot-encoding of categorical values in train dataset
col_dummies = ['ProductCD', 'card4', 'card6', 'P_emaildomain']

for c in col_dummies:
    df_train[c] = pd.get_dummies(df_train[c])
    df_test[c] = pd.get_dummies(df_test[c])
    
# drop one-hot-encoded features     
df_train.drop(columns=col_dummies, inplace = True)  
df_test.drop(columns=col_dummies, inplace = True) 

In [12]:
# after optimization
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Columns: 176 entries, isFraud to V321
dtypes: int16(175), int64(1)
memory usage: 206.1 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 0 to 506690
Columns: 175 entries, TransactionAmt to V321
dtypes: int16(175)
memory usage: 173.0 MB


In [14]:
# free memory
del df_train_ident, df_train_transact, df_test_ident, df_test_transact
gc.collect()

90